In [1]:
## reset specific variables (replace regular_expression by the variables of interest)
#%reset_selective <regular_expression>

# reset all variables
%reset -f

In [2]:
## Importing libraries

from datetime import datetime, date, timedelta
from array import *
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from pylab import savefig
import seaborn as sns
import pandas as pd
import csv
import json
import datetime as dt
from pymongo import MongoClient
from mongoengine import *

%matplotlib inline

In [3]:
## Creating/Connecting Mongo DB instances

# Provide the mongodb atlas url to connect python to mongodb using pymongo
#CONNECTION_STRING = "mongodb+srv://<jgu>:<123>@<cluster-jgu>.mongodb.net/SMARTAttributesFilter"

connect(db='SMARTAttributesFilter', alias='SMARTAttributesFilter_alias')

connect(db='FailuresAppsLocation', alias='FailuresAppsLocation_alias')

connect(db='SMARTAtt_FailuresAppsLocation', alias='SMARTAtt_FailuresAppsLocation_alias')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [4]:
## Setting document schema

class SMARTAtt(Document):
     disk_id = FloatField(required=False, default='0')
     timestamp = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     r_sectors = FloatField(required=False, default='0')
     u_errors = FloatField(required=False, default='0')
     p_failedA = FloatField(required=False, default='0')
     p_failedB = FloatField(required=False, default='0')
     e_failedA = FloatField(required=False, default='0')
     e_failedB = FloatField(required=False, default='0')
     n_b_written = FloatField(required=False, default='0')
     n_b_read = FloatField(required=False, default='0')
     meta = {'db_alias': 'SMARTAttributesFilter_alias'}

class FailuresAppsLocation(Document):
     disk_id = FloatField(required=False, default='0')
     failure_time = DateTimeField(required=False, default='0')
     model_x = StringField(required=False, default='0')
     model_y = StringField(required=False, default='0')
     app = StringField(required=False, default='0')
     node_id = FloatField(required=False, default='0')
     rack_id = FloatField(required=False, default='0')
     machine_room_id = FloatField(required=False, default='0')
     meta = {'db_alias': 'FailuresAppsLocation_alias'}

class SMARTAtt_FailuresAppsLocation(Document):
     smart_att = ReferenceField(SMARTAtt)
     failures_app_location = ReferenceField(FailuresAppsLocation)
     meta = {'db_alias': 'SMARTAtt_FailuresAppsLocation_alias'}

In [ ]:
# Closing the connection to the DB

disconnect(alias='SMARTAttributesFilter_alias')

disconnect(alias='FailuresAppsLocation_alias_alias')

disconnect(alias='SMARTAtt_FailuresAppsLocation_alias')

In [316]:
## Some queries in Mongoengine

## Creating the object related to the whole collection
#failuresAppsLocationTeste = FailuresAppsLocation.objects()

## Collecting the applications
apps = FailuresAppsLocation.objects().distinct("app")
print(apps)

## Looping inside dcuments, which are filtered for the kind o app
#for app_c in apps:
#   dataPerApp = FailuresAppsLocation.objects(app=app_c).filter()
   #dataPerAppToList = list(dataPerApp)
   #dataPerAppDf = pd.DataFrame.from_dict(dataPerAppToList)
     
#   for doc in dataPerApp:
#      arrayForDataPerApp = np.append(arrayForDataPerApp, doc.to_json())

##The collection size
#len(failuresAppsLocationTeste)


## Query if you know something about the document
#testando = FailuresAppsLocation.objects(disk_id="33722").get()


## Query if you know something about the document
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter().limit(20)
#SMARTAttributesTest = SMARTAtt.objects(timestamp="2019-12-31").filter()

## Printing
#for i in SMARTAttributesTest:
#    print(i.timestamp)
    
## Deleting all collection
#failuresAppsLocationTeste.delete() 
  
##Deleting all collection
#for i in failuresAppsLocationTeste:
 # i.delete()

['DAE', 'DB', 'NAS', 'RM', 'SS', 'WPS', 'WS', 'WSM', 'none']


In [ ]:
## Some queries pandas

# Find an specific line based on a specific value from a column
#df_failuresLocationDocumentsJsonMTTF.loc[df_failuresLocationDocumentsJsonMTTF["mtff_node"] == 2] 

In [13]:
## Space for small tests

failuresLocationDocuments = FailuresAppsLocation.objects()

jSon_failuresLocationDocumentsJson = json.loads(failuresLocationDocuments.to_json())
df_failuresLocationDocumentsJson = pd.DataFrame.from_dict(jSon_failuresLocationDocumentsJson) 


dicDateToString = json.dumps(list(df_failuresLocationDocumentsJson['failure_time']))
dicStringToJson = json.loads(dicDateToString)
dicJsonToDf = pd.DataFrame.from_dict(dicStringToJson)
df_failuresLocationDocumentsJson['failure_time'] = dicJsonToDf['$date']



#print(df_failuresLocationDocumentsJson.model_y.unique())



In [14]:
## Calculating MTTF per node, rack and general

df_failuresLocationDocumentsJsonMTTF = df_failuresLocationDocumentsJson

## Value representing: 2018-01-01 00:00:00 (starting date from the experiment)
#1514764800000

#Subctracting by the initial time of the experiment and turning into hours
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.sub(1514764800000)
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.div(1000)
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.div(60)
df_failuresLocationDocumentsJsonMTTF['failure_time'] = df_failuresLocationDocumentsJson.failure_time.div(60)
#df_failuresLocationDocumentsJsonMTTF.head(3)

# Number of unique values by column
#id_apps = df_failuresLocationDocumentsJsonMTTF.app.unique()
#id_nodes = df_failuresLocationDocumentsJsonMTTF.node_id.unique()
#id_racks = nodes = df_failuresLocationDocumentsJsonMTTF.rack_id.unique()
#id_ssds = nodes = df_failuresLocationDocumentsJsonMTTF.disk_id.unique()

#Calculating the MTTFs
df_general = df_failuresLocationDocumentsJsonMTTF['failure_time'].mean()
df_appNode = df_failuresLocationDocumentsJsonMTTF.groupby(['app', 'node_id'])['failure_time'].mean()
df_appNode = pd.DataFrame(df_appNode)
df_appNode.rename(columns = {'failure_time':'mttf_appNode'}, inplace=True)
df_appRack = df_failuresLocationDocumentsJsonMTTF.groupby(['app', 'rack_id'])['failure_time'].mean()
df_appRack = pd.DataFrame(df_appRack)
df_appRack.rename(columns = {'failure_time':'mttf_appRack'}, inplace=True)
df_app = df_failuresLocationDocumentsJsonMTTF.groupby(['app'])['failure_time'].mean()
df_app = pd.DataFrame(df_app)
df_app.rename(columns = {'failure_time':'mttf_app'}, inplace=True)
df_all = df_failuresLocationDocumentsJsonMTTF.groupby(['app', 'rack_id', 'node_id'])['failure_time'].mean()

# Merging the results with the original dataframe
df_result = pd.merge(df_failuresLocationDocumentsJsonMTTF, df_appNode, how='left', on=['app', 'node_id'])
df_result = pd.merge(df_result, df_appRack, how='left', on=['app', 'rack_id'])
df_result = pd.merge(df_result, df_app, how='left', on=['app'])

# Choosing the columns of interest
df_resultFiltered = df_result.loc[:,['app','node_id', 'rack_id','mttf_appNode','mttf_appRack','mttf_app']]
df_resultFiltered.to_csv('failuresLocationApp.csv')
#df_result.head(30)



#df.groupby('Sex').sum().plot(kind='bar');
#print(df_app)
#df_appNode.groupby(['app']).plot(kind='bar')
#df.groupby(['Sex', 'Survived'] )['Survived'].count().plot.bar(figsize=(8, 6));

#d1 = data[data["City"] == "Houston"]
#dataFrame.Reg_Price[i]
#df.loc[df['column_name'] == some_value]



In [15]:
# Investigating Flash technology

flashtech = pd.DataFrame({
'model_y':['A3', 'A2', 'B3' , 'B2', 'C1', 'C2'],  
'flash': ['MLC', 'MLC', 'MLC' , 'MLC', '3D-TLC', '3D-TLC']})

df_flashtech = pd.merge(df_result, flashtech, how='left', on=['model_y'])

df_flashtech = df_flashtech.groupby(['app','flash'])['failure_time'].mean()

df_flashtech = pd.DataFrame(df_flashtech)

df_flashtech.rename(columns = {'failure_time':'mttf_flash'}, inplace=True)

df_flashtech.to_csv('flashtech.csv')
#df_flashtech.head()


In [ ]:
# Investigating Capacity

capacity = pd.DataFrame({
'model_y':['A3', 'A2', 'B3' , 'B2', 'C1', 'C2'],  
'capacity': ['480GB', '800GB', '1920GB' , '1920GB', '1920GB', '960GB']})

df_flashtech = pd.merge(df_result, flashtech, how='left', on=['model_y'])

df_flashtech = df_flashtech.groupby(['app','flash'])['failure_time'].mean()

df_flashtech = pd.DataFrame(df_flashtech)

df_flashtech.rename(columns = {'failure_time':'mttf_flash'}, inplace=True)

df_flashtech.to_csv('flashtech.csv')
#df_flashtech.head()

In [ ]:
##  Creating Graphs

fig = sns.catplot(x="app", y="failure_time", hue="model_x", data=df_failuresLocationDocumentsJson,
               row="node_id", col="rack_id", kind="bar", ci=90, palette="Blues_d", aspect=0.9, height=4.5
                  , legend_out = True, margin_titles = True)

fig.set_axis_labels("App", "MTTF")
fig.set_xticklabels(["DAE", "DB", "NAS", "RM", "SS", "WPS", "WS", "WSM", "none"])

plt.savefig('teste.pdf', dpi=1200)

In [318]:
df_failuresLocationDocumentsJson.to_csv('failuresLocation.csv', index = None, header=True)